In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Eur_Usd'
years = '2021-2022'

In [4]:
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback

def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [5]:
def apply_impact(val, impact):
    return float(val) * float(impact)

In [6]:
currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Previous'], row['Impact']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-42e2058cb015>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
<ipython-input-6-42e2058cb015>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [7]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{years}.csv')
df_long.Date = pd.to_datetime(df_long.Date, utc=True)
df_long.reset_index(drop=True, inplace=True)

df_long['atr'] = atr(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['chop'] = chop(df_long)
df_long['vo'] = vo(df_long['Volume'])

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

In [8]:
df

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,atr,adx,chop,vo,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y
0,2021-04-01 11:30:00+00:00,1.17420,1.17424,1.17351,1.17357,1.17431,1.17436,1.17364,1.17369,1.17426,...,0.000438,14.978938,0.475010,0.112070,1.0,1.0,0.0,1.0,1.0,0.0
1,2021-04-01 11:35:00+00:00,1.17358,1.17394,1.17350,1.17371,1.17370,1.17406,1.17363,1.17384,1.17364,...,0.000428,13.915372,0.524667,0.057478,1.0,1.0,0.0,1.0,1.0,0.0
2,2021-04-01 11:40:00+00:00,1.17372,1.17372,1.17349,1.17357,1.17385,1.17385,1.17362,1.17369,1.17378,...,0.000409,12.935083,0.507852,0.015284,1.0,1.0,0.0,1.0,1.0,0.0
3,2021-04-01 11:45:00+00:00,1.17358,1.17390,1.17357,1.17368,1.17370,1.17402,1.17369,1.17382,1.17364,...,0.000406,12.061841,0.533390,0.002524,1.0,1.0,0.0,1.0,1.0,0.0
4,2021-04-01 11:50:00+00:00,1.17367,1.17382,1.17353,1.17382,1.17381,1.17395,1.17366,1.17395,1.17374,...,0.000399,11.696054,0.608498,-0.074777,1.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74712,2022-04-01 05:35:00+00:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,...,0.000272,9.813219,0.620954,0.050204,1.0,0.0,0.0,1.0,0.0,0.0
74713,2022-04-01 05:40:00+00:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,...,0.000299,9.807821,0.517246,0.096511,1.0,0.0,0.0,1.0,0.0,0.0
74714,2022-04-01 05:45:00+00:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,...,0.000289,10.947864,0.532878,0.007229,1.0,0.0,0.0,1.0,0.0,0.0
74715,2022-04-01 05:50:00+00:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,...,0.000294,12.020967,0.535579,-0.022269,1.0,0.0,0.0,1.0,0.0,0.0


In [9]:
df_long

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,adx,chop,vo
0,2021-01-04 12:00:00+00:00,1.22925,1.23056,1.22864,1.23028,1.22937,1.23070,1.22878,1.23042,1.22931,1.23062,1.22871,1.23035,5837,0.001505,77.855402,0.390399,0.104962
1,2021-01-04 13:00:00+00:00,1.23030,1.23045,1.22856,1.22982,1.23043,1.23058,1.22870,1.22997,1.23036,1.23052,1.22864,1.22990,7722,0.001484,72.574559,0.426459,0.142061
2,2021-01-04 14:00:00+00:00,1.22984,1.23089,1.22830,1.22885,1.22997,1.23108,1.22844,1.22900,1.22990,1.23098,1.22837,1.22892,9625,0.001590,69.976608,0.439453,0.183780
3,2021-01-04 15:00:00+00:00,1.22886,1.22902,1.22596,1.22712,1.22902,1.22916,1.22613,1.22724,1.22894,1.22908,1.22605,1.22718,10731,0.001694,66.434238,0.471597,0.203536
4,2021-01-04 16:00:00+00:00,1.22708,1.22767,1.22519,1.22532,1.22725,1.22782,1.22532,1.22544,1.22716,1.22774,1.22526,1.22538,9301,0.001806,53.789532,0.496042,0.173497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,2021-12-31 17:00:00+00:00,1.13782,1.13857,1.13766,1.13856,1.13797,1.13871,1.13781,1.13870,1.13790,1.13864,1.13774,1.13863,1729,0.001784,17.292895,0.417353,0.090819
6218,2021-12-31 18:00:00+00:00,1.13855,1.13855,1.13741,1.13840,1.13869,1.13869,1.13756,1.13854,1.13862,1.13862,1.13748,1.13847,828,0.001841,19.675722,0.429303,-0.059084
6219,2021-12-31 19:00:00+00:00,1.13842,1.13852,1.13791,1.13820,1.13856,1.13865,1.13805,1.13834,1.13849,1.13858,1.13798,1.13827,619,0.001769,21.590350,0.414306,-0.182146
6220,2021-12-31 20:00:00+00:00,1.13819,1.13845,1.13749,1.13749,1.13832,1.13863,1.13767,1.13767,1.13826,1.13854,1.13758,1.13758,623,0.001785,23.330537,0.417657,-0.268561


In [10]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [13]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]

        if use_long:
            if len(df_long.loc[df_long.Date >= curr_date]) == 0:
                break

            curr_long = df_long.loc[df_long.Date <= curr_date]
            curr_long.reset_index(drop=True, inplace=True)

            if len(curr_long) < 2:
                continue

            actual_base, actual_counter = df.loc[df.index[i - 1], ['Actual_Class_x', 'Actual_Class_y']]
            volume, chop, adx = curr_long.loc[curr_long.index[-2], ['vo', 'chop', 'adx']]

        else:
            volume, chop, adx, actual_base, actual_counter = df.loc[df.index[i - 1], ['vo', 'chop', 'adx', 'Actual_Class_x', 'Actual_Class_y']]
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
        sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        volume_signal = volume >= volume_cutoff if volume_cutoff is not None else True
        chop_signal = chop <= chop_cutoff if chop_cutoff is not None else True
        adx_signal = adx >= adx_cutoff if adx_cutoff is not None else True
        news_buy = actual_base > 0 if use_news else True
        news_sell = actual_base < 0 if use_news else True

        buy_signal = buy_signal and volume_signal and chop_signal and adx_signal and news_buy
        sell_signal = sell_signal and volume_signal and chop_signal and adx_signal and news_sell

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [14]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10]
n_bars_vals = [3]
pip_movement_vals = [20]
use_pullback_vals = [True]
pips_to_risk_vals = ['bars', 10, 20]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.35, 0.50]
# NEW STUFF
volume_cutoffs = [None, 0.0, 0.10, 0.15, 0.20]  # Cutoff for VO indicator - values should be between 0 and 1, higher is more restrictive
chop_cutoffs = [None, 0.35, 0.50]  # Cutoff for chop indicator - values should be between 0 and 1, lower is more restrictive
adx_cutoffs = [None, 20, 25, 30]  # Cutoff for adx indicator - values should be between 0 and 100, higher is more restrictive
use_news_vals = [True, False]  # Whether or not to check the news when making a trade
use_long_vals = [True, False]  # Whether or not to use a longer timeframe when looking at VO, chop, and adx

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals) * len(pips_to_risk_vals) * len(pullback_percentages) * len(volume_cutoffs) * len(chop_cutoffs) * len(adx_cutoffs) * len(use_news_vals) * len(use_long_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pips_to_risk in pips_to_risk_vals:
                        for pullback_percentage in pullback_percentages:
                            for volume_cutoff in volume_cutoffs:
                                for chop_cutoff in chop_cutoffs:
                                    for adx_cutoff in adx_cutoffs:
                                        for use_news in use_news_vals:
                                            for use_long in use_long_vals:
                                                all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_volume_cutoff = None
best_chop_cutoff = None
best_adx_cutoff = None
best_use_news_val = None
best_use_long_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'volume_cutoff': volume_cutoff, 'chop_cutoff': chop_cutoff, 'adx_cutoff': adx_cutoff, 'use_news': use_news, 'use_long': use_long})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_volume_cutoff = volume_cutoff
        best_chop_cutoff = chop_cutoff
        best_adx_cutoff = adx_cutoff
        best_use_news_val = use_news
        best_use_long_val = use_long

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1440



KeyboardInterrupt: 

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best volume cutoff: ' + str(best_volume_cutoff))
print('Best chop cutoff: ' + str(best_chop_cutoff))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best use news val: ' + str(best_use_news_val))
print('Best use long val: ' + str(best_use_long_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 349.98600000000033
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 5
Best pip movement val: 30
Best use pullback val: True
Best use friday val: True
Best wait period: 2
Best pips to risk: 20
Best pullback percentage: 0.3
Best ema: ema400
Best invert val: False
-----------------------
Top results:
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True, 'use_friday': True, 'wait_period': 4, 'pips_to_risk': 10, 'pullback_percentage': 0.5, 'ema': None, 'invert': True}
{'reward': 349, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True, 'use_friday': True, 'wait_period': 2, 'pips_to_risk': 20, 'pullback_percentage': 0.3, 'ema': 'ema400', 'invert': False}
